# DNA Dynamics
In this notebook, we use AmberTools and OpenMM to perform a quick molecular dynamics simulation of a small DNA duplex.

In [ ]:
%matplotlib notebook
from matplotlib.pyplot import *
import moldesign as mdt
from moldesign import units as u
mdt.config.print_configuration()

### Unit system
To begin, go ahead and select your preferred unit system.

In [ ]:
mdt.units.default.energy = u.kcalpermol
mdt.units.default.length = u.nm
mdt.units.default.time = u.ps

## Simulation parameters
All integrators and models can tell you about the parameters they accept - let's take a look at the OpenMM methods.

In [ ]:
print 'OpenMMPotential:'
print mdt.models.OpenMMPotential.print_parameters()
print '\nOpenMMIntegrator:'
print mdt.integrators.OpenMMLangevin.print_parameters()

## Prepping the structure
Let's download a DNA crystal structure and get it ready to simulate. First, we'll download [PDB 1BNA](http://www.rcsb.org/pdb/explore/jmol.do?structureId=1BNA&bionumber=1) and visualize it.

Click on an atom in the 3D view to get more information about it.

In [ ]:
xtal_structure = mdt.read('data/1BNA.pdb')
viewer = xtal_structure.draw()
viewer

As you can see, we've got a nice strand of DNA with a bunch of not-so-nice water molecules surrounding it. We'll remove those before going any farther.

In [ ]:
bare_dna = mdt.Molecule([atom for atom in xtal_structure.atoms
                        if atom.residue.type != 'water'], copy_atoms=True)
bare_dna.draw()

### Set up forcefield
Next, we'll assign forcefield parameters to our system. This will also add any missing hydrogens to the molecule.

Notice that, while this runs, a tab named "tleap" will appear under the cell. Go ahead and click on this - it indicates that the tleap command was run remotely using Docker. Especially during "production", it's important to examine the program's logs and understand the meaning of any warnings or errors it emits.

In [ ]:
mol = mdt.assign_forcefield(bare_dna)
mol.draw()

### Set up geometry constraints
Because this a very small piece of DNA, it's likely to unravel as we simulate it. To keep this from happening, let's **freeze the terminal base pairs on each strand**.

**Click on the base pairs to be constrained:**

In [ ]:
rs = mdt.ui.ResidueSelector(mol)
rs

In [ ]:
for atom in rs.selected_atoms:
    mol.constrain_atom(atom)

Of course, fixing the positions of the terminal base pairs is a fairly extreme step. For extra credit, see if you can find a less heavy-handed keep the terminal base pairs bonded. (Try using tab-completion to see what other constraint methods are available)

## Simulation
We're set up - let's run dynamics! We'll:
 1. Add an OpenMM energy model and a Langevin integrator to our molecule
 1. run a short energy minimization to remove any large instabilities, and
 1. Run 50 ps of dynamics

In [ ]:
model = mdt.models.OpenMMPotential(implicit_solvent='obc',
                                  cutoff=5.0*u.angstrom)
integrator = mdt.integrators.OpenMMLangevin(timestep=2.0*u.fs, constraints=['hbonds'],
                                          temperature=300.0*u.kelvin,
                                          frame_interval=1.0*u.ps)
mol.set_energy_model(model)
mol.set_integrator(integrator)

The parameters for the energy model, integrator, and constraints can be reviewed (and modified) at any time:

In [ ]:
model.params

In [ ]:
integrator.params

In [ ]:
for constraint in mol.constraints: print constraint

Nearly every MD simulation should be preceded by an energy minimization, especially for crystal structure data. This will remove any energetically catastrophic clashes between atoms and prevent our simulation from blowing up.

In [ ]:
trajectory = mol.minimize(nsteps=200)

In [ ]:
trajectory = mol.minimize(nsteps=200)
plot(trajectory.potential_energy)
xlabel('steps');ylabel('energy / %s'%trajectory.potential_energy.units)
title('Energy relaxation')
grid()
viz = trajectory.draw3d()
viz.licorice()
viz

### Dynamics
We're ready to run 50 picoseconds of dynamics at room temperature (that's 300º Kelvin). This will probably take a few minutes - if you're on an especially pokey computer, you might want to reduce the length of the simulation.

In [ ]:
integrator.params.frame_interval=500
traj = mol.run(run_for=50.0*u.ps)

## Analysis
Let's look at the results of our trajectory. The trajectory object (named `traj`) gives us direct access to the timeseries data:

In [ ]:
figure()
plot(traj.time, traj.kinetic_energy, label='kinetic energy')
plot(traj.time, traj.potential_energy - traj.potential_energy[0], label='potential_energy')
xlabel('time / {time.units}'.format(time=traj.time))
ylabel('energy / {energy.units}'.format(energy=traj.kinetic_energy))
grid()
legend(loc='center right')

figure()
# Using the trajectory's 'plot' method will autogenerate axes labels with the appropriate units
traj.plot('time','kinetic_temperature')

Of course, it's always nice to see the 3D structure:

In [ ]:
debug

In [ ]:
viewer = traj.draw3d()
viewer

If you examine the dynamics, you'll likely observe one or more base pairs flipping out of the double helix. They'll be different every time because the simulation uses random numbers to generate the thermal fluctuations.

In the next cell, try plotting the RMSDs of a few base pairs. You can get the residue's names by clicking on them in the trajectory animation.

In [ ]:
res0 = mol.atoms[679].residue
res1 = mol.atoms[142].residue
res2 = mol.atoms[100].residue

figure()
plot(traj.time, traj.rmsd(), lw=2, label='overall rmsd')
plot(traj.time, traj.rmsd(res0.atoms), label='fixed residue')
plot(traj.time, traj.rmsd(res1.atoms), label='flipped residue')
plot(traj.time, traj.rmsd(res2.atoms), label='unflipped residue')
grid()
xlabel('time / fs')
ylabel(u'rmsd / {}'.format(u.default.length))
legend(loc='upper left')